In [ ]:
import requests
import time
import pandas as pd
pd.set_option("display.max.columns", None)

## Congress
headers = {'X-API-Key': 'Li9aCR2BjLZviTE2vhRXQ9QDqq1XU8NxXUXkQDbA',}
pp_base = 'https://api.propublica.org/congress/v1/'

# GET https://api.propublica.org/congress/v1/{congress}/bills/{bill-id}.json

CONGRESS = 114

bill_id = 'sres642'

data = requests.get(f'{pp_base}{CONGRESS}/bills/{bill_id}.json', headers = headers).json()
data

In [ ]:
data['results'][0]['summary']

In [ ]:
# Read the CSV file into a pandas dataframe
dfd = pd.read_csv('bills_database/bills_93-114_dem.csv')
dfr = pd.read_csv('bills_database/bills_93-114_rep.csv')
minor = pd.read_csv('bills_database/minor.csv', header=None)
major = pd.read_csv('bills_database/major.csv')

# Create code dictionaries
minor[['code', 'description']] = minor[0].str.split(":", n=1, expand=True)
minor = minor.drop(0, axis=1)
minor = minor.set_index('code').to_dict('index')
minor = {i:val['description'] for i, val in minor.items()}
major = major.set_index('code').to_dict('index')
major = {i:val['description'] for i, val in major.items()}

# Display the dataframe
df = pd.concat([dfd, dfr])

# Propublica only has info from congress 113
df = df.loc[df['Cong'] >= 113]


In [ ]:
df['idx'] = df['BillType'] + df['BillNum'].astype(str)
dict_bills = df.to_dict('records')

In [ ]:
def mergedicts(dict1, dict2):
    '''
    Esta funcion ayuda a combinar diccionarios
    '''
    for k in set(dict1.keys()).union(dict2.keys()):
        if k in dict1 and k in dict2:
            if isinstance(dict1[k], dict) and isinstance(dict2[k], dict):
                yield (k, dict(mergedicts(dict1[k], dict2[k])))
            else:
                yield (k, dict2[k])
        elif k in dict1:
            yield (k, dict1[k])
        else:
            yield (k, dict2[k])

final_dta = []
error_aux = []
error = []

for i in dict_bills:
    congress = i['Cong']
    bill_id = i['idx']
    if congress == 113 and bill_id == 'hr1771':
        continue
    try:
        try:
            propublica_data = requests.get(f'{pp_base}{congress}/bills/{bill_id}.json', headers = headers).json()
        except: 
            error.append(tuple(congress, bill_id, False))
            continue
    except:
        time.sleep(75)
        try:
            propublica_data = requests.get(f'{pp_base}{congress}/bills/{bill_id}.json', headers = headers).json()
        except:
            error.append(tuple(congress, bill_id, False))
            continue
    try:
        if propublica_data['status'] == 'ERROR': 
            error.append(tuple(congress, bill_id, True))
            continue
        data = dict(propublica_data['results'][0])
        if data['summary'] == '': continue
        data['Major_desc'] = major.get(i['Major'], None)
        data['Minor_desc'] = major.get(i['Minor'], None)
        final_dta.append(dict(mergedicts(i,data)))
    except:
        print(propublica_data)
        error_aux.append(propublica_data)
        continue

In [ ]:
data_final = pd.DataFrame(final_dta)
data_final.to_pickle("df.pkl")

In [ ]:
data_final = pd.read_pickle("data_final.pkl")
data_final